In [167]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime, timedelta
from sklearn.datasets import make_regression
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [168]:
home = raw_input("Which Home?")
file_path = 'Year1/Home' + home + '_yr1.csv'

Which Home?2


In [169]:
print(file_path)
df = pandas.read_csv(file_path)
def datetime_range(start, end, delta):
    end += 4*delta
    current = start
    while current < end:
        yield current
        current += delta

dts = [dt for dt in 
       datetime_range(datetime(2018, 1, 1, 0), datetime(2018, 12 , 31 , 23), 
       timedelta(minutes=15))]
df['date/time'] = dts

Year1/Home2_yr1.csv


In [170]:
for i, row in df.iterrows():
#handling seasons
    df.at[i, 'Date'] = row[1].day
    df.at[i, 'Month'] = row[1].month
    df.at[i, 'Hour'] = row[1].hour
    df.at[i, 'Minute'] = row[1].minute
    if row[1].month in range(1,4) :
        df.at[i, 'Summer'] = 1
    elif row[1].month in range(4,7):
        df.at[i, 'Spring'] = 1
    elif row[1].month in range(7,10):
        df.at[i, 'Fall'] = 1
    elif row[1].month in range(10,13):
        df.at[i, 'Winter'] = 1
        
#handling peak hours   
    if row[1].hour in [0,1,2,3,4,5,6,7,20,21,22,23]:
        df.at[i, 'Peak Hour'] = 1
    else:
        df.at[i, 'Peak Hour'] = 0
        
#handling holidays & weekends
    if row[1].weekday() in [5,6] :
        df.at[i, 'Holiday'] = 1
    elif row[1].month == 10 and row[1].day == 2:
        df.at[i, 'Holiday'] = 1
    elif row[1].month == 1 and row[1].day == 26:
        df.at[i, 'Holiday'] = 1
    else:
        df.at[i, 'Holiday'] = 0

df['Spring'].fillna(0,inplace=True)
df['Fall'].fillna(0,inplace=True)
df['Summer'].fillna(0,inplace=True)
df['Winter'].fillna(0,inplace=True)

In [217]:
month = raw_input("Enter the Month:")
date = raw_input("Enter the Date:")
hour = raw_input("Enter the Hour:")
minute = raw_input("Enter the Minute:")
dt = datetime(2018, int(month), int(date), int(hour), int(minute) , 0)
print(dt)

Enter the Month:9
Enter the Date:17
Enter the Hour:0
Enter the Minute:0
2018-09-17 00:00:00


In [218]:
for i, row in df.iterrows():
    if row[1] == dt:
        break
x_train = df[['Date','Month','Hour','Minute','Summer','Winter', 'Fall','Spring' , 'Holiday', 'Peak Hour']][:i]
y_train = df[['Consumption']][:i]
x_test = df[['Date','Month','Hour','Minute','Summer','Winter', 'Fall','Spring' , 'Holiday', 'Peak Hour']] [i:96+i]
y_test = df[['Consumption']][i:96+i]

In [223]:
regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)
y_pred_r = regr.predict(x_test)
print("Mean squared error LR: %.9f"
      % mean_squared_error(y_test,y_pred_r))
y_pred_r = pd.DataFrame({'Predicted Consumption':y_pred_r[:,0]}, index=y_test.index) 
dur = [dt for dt in 
       datetime_range(datetime(2018, int(month), int(date), int(hour), int(minute)), datetime(2018, 12 , 31 , 23), 
       timedelta(minutes=15))]
dur = dur[0:96]
y_pred_r['Actual Consumption'] = y_test
y_pred_r['Date/Time'] = dur
display(y_pred_r)

Mean squared error LR: 1.361366079


,Predicted Consumption,Actual Consumption,Date/Time
24864,0.649865,0.134160,2018-09-17 00:00:00
24865,0.640176,0.126210,2018-09-17 00:15:00
24866,0.630487,0.017903,2018-09-17 00:30:00
24867,0.620799,0.018029,2018-09-17 00:45:00
24868,0.675743,0.019744,2018-09-17 01:00:00
24869,0.666054,0.498150,2018-09-17 01:15:00
24870,0.656365,3.774200,2018-09-17 01:30:00
24871,0.646677,0.701340,2018-09-17 01:45:00
24872,0.701621,0.612250,2018-09-17 02:00:00
24873,0.691932,0.764670,2018-09-17 02:15:00


In [224]:
regr = KNeighborsRegressor(n_neighbors=1)
regr.fit(x_train, y_train)
y_pred_r = regr.predict(x_test)
#print(y_pred_r)
print("Mean squared error LR: %.9f"
      % mean_squared_error(y_test,y_pred_r))
y_pred_r = pd.DataFrame({'Predicted Consumption':y_pred_r[:,0]}, index=y_test.index) 
dur = [dt for dt in 
       datetime_range(datetime(2018, int(month), int(date), int(hour), int(minute)), datetime(2018, 12 , 31 , 23), 
       timedelta(minutes=15))]
dur = dur[0:96]
y_pred_r['Actual Consumption'] = y_test
y_pred_r['Date/Time'] = dur
display(y_pred_r)

Mean squared error LR: 2.186636258


,Predicted Consumption,Actual Consumption,Date/Time
24864,0.377270,0.134160,2018-09-17 00:00:00
24865,0.402250,0.126210,2018-09-17 00:15:00
24866,0.429530,0.017903,2018-09-17 00:30:00
24867,0.355390,0.018029,2018-09-17 00:45:00
24868,0.413060,0.019744,2018-09-17 01:00:00
24869,0.534690,0.498150,2018-09-17 01:15:00
24870,0.849140,3.774200,2018-09-17 01:30:00
24871,3.198200,0.701340,2018-09-17 01:45:00
24872,2.989800,0.612250,2018-09-17 02:00:00
24873,0.527070,0.764670,2018-09-17 02:15:00


In [225]:
regr = AdaBoostRegressor(base_estimator=None, n_estimators=50, learning_rate=1.0, loss='linear', random_state=None)
regr.fit(x_train, y_train)
y_pred_r = regr.predict(x_test)
print("Mean squared error LR: %.9f"
      % mean_squared_error(y_test,y_pred_r))
# y_pred_r = pd.DataFrame({'Predicted Consumption':y_pred_r[:,0]}, index=y_test.index) 
# dur = [dt for dt in 
#        datetime_range(datetime(2018, int(month), int(date), int(hour), int(minute)), datetime(2018, 12 , 31 , 23), 
#        timedelta(minutes=15))]
# dur = dur[0:96]
# y_pred_r['Actual Consumption'] = y_test
# y_pred_r['Date/Time'] = dur
# display(y_pred_r)

Mean squared error LR: 1.412205172


In [226]:
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x_train, y_train)
y_pred_r = regr.predict(x_test)
print("Mean squared error LR: %.9f"
      % mean_squared_error(y_test,y_pred_r))
# y_pred_r = pd.DataFrame({'Predicted Consumption':y_pred_r[:,0]}, index=y_test.index) 
# dur = [dt for dt in 
#        datetime_range(datetime(2018, int(month), int(date), int(hour), int(minute)), datetime(2018, 12 , 31 , 23), 
#        timedelta(minutes=15))]
# dur = dur[0:96]
# y_pred_r['Actual Consumption'] = y_test
# y_pred_r['Date/Time'] = dur
# display(y_pred_r)

Mean squared error LR: 1.358031982


/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
